In [1]:
import numpy as np
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
from mmdet.registry import VISUALIZERS
import mmcv
!pwd

/home/ubuntu/mmdetection


In [2]:
register_all_modules()
checkpoint_file = './work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_balloon/epoch_12.pth'
config_file = './work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_balloon/mask-rcnn_r50-caffe_fpn_ms-poly-1x_balloon.py'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

02/10 10:25:49 - mmengine - WARNING - The "model" registry in mmdet did not set import location. Fallback to call `mmdet.utils.register_all_modules` instead.
02/10 10:25:49 - mmengine - WARNING - The "task util" registry in mmdet did not set import location. Fallback to call `mmdet.utils.register_all_modules` instead.
Loads checkpoint by local backend from path: ./work_dirs/mask-rcnn_r50-caffe_fpn_ms-poly-1x_balloon/epoch_12.pth
02/10 10:25:49 - mmengine - WARNING - The "dataset" registry in mmdet did not set import location. Fallback to call `mmdet.utils.register_all_modules` instead.
02/10 10:25:49 - mmengine - WARNING - The "transform" registry in mmdet did not set import location. Fallback to call `mmdet.utils.register_all_modules` instead.


#### 推理并生成gif

In [3]:
import cv2
import imageio
import torch
from mmengine.utils import track_iter_progress


def inference_video(video_path):
	video_reader = mmcv.VideoReader(video_path)
	all_frames = []
	for img in track_iter_progress(video_reader):
		result = inference_detector(model, img)
		masks = result.pred_instances.masks
		masks_int = torch.tensor(masks, dtype=int).cpu().numpy()  # torch.Size([bboxes, 1080, 1920])
		mask = masks_int.sum(axis=0)
		mask = mask > 0
		background = np.logical_not(mask)   # [1, 1080, 1920]
		img[0][0] # [71, 87, 94] (1080, 1920, 3)
		# 利用广播机制
		mask = mask.reshape(1080, 1920, 1)
		mask_img = mask * img
		mask_img = mask_img.reshape(1080, 1920, 3)
		mask_img = mmcv.rgb2bgr(mask_img)
		gray_img = mmcv.bgr2gray(img) * background
		mask_img += gray_img.reshape(1080, 1920, 1)
		mask_img = cv2.resize(mask_img, [240, 135])
		all_frames.append(mask_img)
	return all_frames

def frame_to_gif(frame_list):
	gif = imageio.mimsave('./test_result.gif', frame_list, 'GIF', duration=0.0005)  

frame_list = inference_video('./data/balloon/test_video.mp4')
frame_to_gif(frame_list)

[                                 ] 1/150, 0.6 task/s, elapsed: 2s, ETA:   247s

/tmp/ipykernel_11724/1579105893.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masks_int = torch.tensor(masks, dtype=int).cpu().numpy()  # torch.Size([bboxes, 1080, 1920])


[>>>>>>>>>>>>>>>>>>>>>>>>>     ] 125/150, 3.7 task/s, elapsed: 34s, ETA:     7s
